In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer

from scipy import stats
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

import statsmodels.api as sm
from linearmodels.panel import PooledOLS

In [38]:
df = pd.read_pickle("data_after_normalization-v1.pickle")

In [39]:
df = df.drop(['Charging_Time_in_hours', 'Precipitation','Temperature','pd','Bahnhof','Restaurant'
              ,'Biergarten','Community_Centre','Hospital','Kino','Parking','university','Charging_time_in_hours_log_New'], axis = 1)

In [40]:
df = df.rename(columns = {'Temperature_New':'Temperature', 'Precipitation_New':'Precipitation',
                          'pd_New':'pd','Bahnhof_New':'Bahnhof','Restaurant_New':'Restaurant','Biergarten_New':'Biergarten',
                          'Community_Centre_New':'Community_Centre', 'Hospital_New':'Hospital',
                         'Kino_New':'Kino', 'Parking_New':'Parking','university_New':'university', 'Charging_Time_in_hours_New':'Charging_Time_in_hours',
                          })

In [41]:
df['Year'] = pd.to_datetime(df['datetime_start'], format='%Y')

In [42]:
df['Month'] = pd.to_datetime(df['datetime_start'], format='%M')

In [43]:
df['Day'] = pd.to_datetime(df['datetime_start'], format='%D')

In [44]:
df.info()

In [45]:
mod = smf.ols(formula='Charging_time_in_hours_log ~ Temperature + pd+ Precipitation + clc_code', data=df)

In [46]:
res = mod.fit()

In [47]:
print(res.summary())

In [48]:
res_cat = smf.ols(formula='Charging_time_in_hours_log ~ Temperature + pd+ Precipitation+ C(clc_code)', data=df).fit()

In [49]:
print(res_cat.params)

In [50]:
res1 = smf.ols(formula='Charging_time_in_hours_log ~ Temperature + pd+ Precipitation+ C(clc_code) -1 ', data=df).fit()

In [51]:
print(res1.params)

## MUltiplicative 

In [52]:
res2 = smf.ols(formula='Charging_time_in_hours_log ~ Temperature + Temperature*C(clc_code) + Temperature*C(clc_code)*C(year)  +  pd + pd*C(clc_code) +pd*C(clc_code)*C(year) + Precipitation+ Precipitation*C(clc_code)+Precipitation*C(clc_code)*C(year)- 1', data=df).fit()

In [53]:
print(res2.summary())

In [55]:
res2 = smf.ols(formula='Charging_time_in_hours_log ~ Temperature+ Temperature*pd + Temperature*C(clc_code) + Temperature*C(clc_code)*C(year) + pd + pd*C(clc_code) +pd*C(clc_code)*C(year)+Temperature*pd*C(clc_code)*C(year) +Biergarten +pd*Biergarten +C(clc_code)*pd*Temperature*Biergarten +pd*Temperature+ Community_Centre +pd*Community_Centre +C(clc_code)*pd*Temperature*Community_Centre +pd*Temperature*Community_Centre', data=df).fit()

In [56]:
print(res2.summary())

In [57]:
import sys
filename  = open("res2",'w')
sys.stdout = filename
print(res2)

In [34]:
print(res2.params)

Intercept                                                     1.452230
C(clc_code)[T.Suburban]                                      -0.103077
C(clc_code)[T.Uninhabited]                                   -1.302255
C(clc_code)[T.Urban]                                          0.200462
C(year)[T.2020]                                              -0.175147
                                                                ...   
C(clc_code)[T.Urban]:Temperature:Community_Centre             0.002517
pd:Temperature:Community_Centre                               0.000123
C(clc_code)[T.Suburban]:pd:Temperature:Community_Centre      -0.000093
C(clc_code)[T.Uninhabited]:pd:Temperature:Community_Centre    0.027719
C(clc_code)[T.Urban]:pd:Temperature:Community_Centre         -0.000234
Length: 80, dtype: float64


In [50]:
print(res2.summary())

                                OLS Regression Results                                
Dep. Variable:     Charging_time_in_hours_log   R-squared:                       0.019
Model:                                    OLS   Adj. R-squared:                  0.019
Method:                         Least Squares   F-statistic:                     732.1
Date:                        Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                                19:02:32   Log-Likelihood:            -3.6448e+06
No. Observations:                     1796283   AIC:                         7.290e+06
Df Residuals:                         1796235   BIC:                         7.290e+06
Df Model:                                  47                                         
Covariance Type:                    nonrobust                                         
                                                               coef    std err          t      P>|t|      [0.025      0.975]
-----

In [20]:
res_temp = smf.ols(formula='Charging_time_in_hours_log ~ Temperature+Temperature*Restaurant+Temperature*Biergarten+Temperature*Community_Centre+Temperature*Kino+Temperature*Hospital+Temperature*Parking+Temperature*university+Temperature*university+Temperature*Bahnhof', data=df).fit()

In [22]:
print(res_temp.summary())

                                OLS Regression Results                                
Dep. Variable:     Charging_time_in_hours_log   R-squared:                       0.004
Model:                                    OLS   Adj. R-squared:                  0.004
Method:                         Least Squares   F-statistic:                     392.5
Date:                        Tue, 25 Jan 2022   Prob (F-statistic):               0.00
Time:                                16:06:52   Log-Likelihood:            -3.6586e+06
No. Observations:                     1796283   AIC:                         7.317e+06
Df Residuals:                         1796265   BIC:                         7.317e+06
Df Model:                                  17                                         
Covariance Type:                    nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [63]:
res_pd = smf.ols(formula='Charging_time_in_hours_log ~ pd+pd*Restaurant+pd*Biergarten+pd*Community_Centre+pd*Kino+pd*Hospital+pd*Parking+pd*university+pd*university+pd*Bahnhof', data=df).fit()

In [64]:
print(res_pd.summary())

                                OLS Regression Results                                
Dep. Variable:     Charging_time_in_hours_log   R-squared:                       0.010
Model:                                    OLS   Adj. R-squared:                  0.010
Method:                         Least Squares   F-statistic:                     1096.
Date:                        Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                                19:44:21   Log-Likelihood:            -3.6526e+06
No. Observations:                     1796283   AIC:                         7.305e+06
Df Residuals:                         1796265   BIC:                         7.305e+06
Df Model:                                  17                                         
Covariance Type:                    nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------

In [65]:
res_Precipitation = smf.ols(formula='Charging_time_in_hours_log ~ Precipitation+Precipitation*Restaurant+Precipitation*Biergarten+Precipitation*Community_Centre+Precipitation*Kino+Precipitation*Hospital+Precipitation*Parking+Precipitation*university+Precipitation*university+Precipitation*Bahnhof', data=df).fit()

In [66]:
print(res_Precipitation.summary())

                                OLS Regression Results                                
Dep. Variable:     Charging_time_in_hours_log   R-squared:                       0.002
Model:                                    OLS   Adj. R-squared:                  0.002
Method:                         Least Squares   F-statistic:                     171.9
Date:                        Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                                19:46:11   Log-Likelihood:            -3.6604e+06
No. Observations:                     1796283   AIC:                         7.321e+06
Df Residuals:                         1796265   BIC:                         7.321e+06
Df Model:                                  17                                         
Covariance Type:                    nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [67]:
res_clc = smf.ols(formula='Charging_time_in_hours_log ~ C(clc_code)+C(clc_code)*Restaurant+C(clc_code)*Biergarten+C(clc_code)*Community_Centre+C(clc_code)*Kino+C(clc_code)*Hospital+C(clc_code)*Parking+C(clc_code)*university+C(clc_code)*university+C(clc_code)*Bahnhof', data=df).fit()

In [68]:
print(res_clc.summary())

                                OLS Regression Results                                
Dep. Variable:     Charging_time_in_hours_log   R-squared:                       0.010
Model:                                    OLS   Adj. R-squared:                  0.010
Method:                         Least Squares   F-statistic:                     505.3
Date:                        Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                                20:21:24   Log-Likelihood:            -3.6531e+06
No. Observations:                     1796283   AIC:                         7.306e+06
Df Residuals:                         1796247   BIC:                         7.307e+06
Df Model:                                  35                                         
Covariance Type:                    nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------

In [80]:
#mod = ols("Charging_time_in_hours_log ~ C(pd, clc_code)", data=df)